<a href="https://colab.research.google.com/github/Abdullah-bin-hilabi/Opencv-projects/blob/main/CarPricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import kagglehub
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
path = kagglehub.dataset_download("hellbuoy/car-price-prediction")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'car-price-prediction' dataset.
Path to dataset files: /kaggle/input/car-price-prediction


In [ ]:
data = pd.read_csv(path + "/CarPrice_Assignment.csv")

In [ ]:
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [ ]:
data.shape

(205, 26)

In [ ]:
data.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [ ]:
data = pd.get_dummies(data, drop_first=True)

In [ ]:
print(data.columns.tolist())

['car_ID', 'symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price', 'CarName_alfa-romero Quadrifoglio', 'CarName_alfa-romero giulia', 'CarName_alfa-romero stelvio', 'CarName_audi 100 ls', 'CarName_audi 100ls', 'CarName_audi 4000', 'CarName_audi 5000', 'CarName_audi 5000s (diesel)', 'CarName_audi fox', 'CarName_bmw 320i', 'CarName_bmw x1', 'CarName_bmw x3', 'CarName_bmw x4', 'CarName_bmw x5', 'CarName_bmw z4', 'CarName_buick century', 'CarName_buick century luxus (sw)', 'CarName_buick century special', 'CarName_buick electra 225 custom', 'CarName_buick opel isuzu deluxe', 'CarName_buick regal sport coupe (turbo)', 'CarName_buick skyhawk', 'CarName_buick skylark', 'CarName_chevrolet impala', 'CarName_chevrolet monte carlo', 'CarName_chevrolet vega 2300', 'CarName_dodge challenger se', 'CarName_dodge colt (sw)', 'CarName_dodge colt hardtop', 'CarName_dod

In [ ]:
important_columns = [
    'symboling', 'fueltype_gas', 'aspiration_turbo', 'wheelbase', 'curbweight',
    'enginesize', 'horsepower', 'citympg', 'highwaympg', 'boreratio',
    'stroke', 'compressionratio', 'peakrpm'
]

x = data[important_columns]
y = data['price']

In [ ]:
x.describe()

,symboling,wheelbase,curbweight,enginesize,horsepower,citympg,highwaympg,boreratio,stroke,compressionratio,peakrpm
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,2555.565854,126.907317,104.117073,25.219512,30.751220,3.329756,3.255415,10.142537,5125.121951
std,1.245307,6.021776,520.680204,41.642693,39.544167,6.542142,6.886443,0.270844,0.313597,3.972040,476.985643
min,-2.000000,86.600000,1488.000000,61.000000,48.000000,13.000000,16.000000,2.540000,2.070000,7.000000,4150.000000
25%,0.000000,94.500000,2145.000000,97.000000,70.000000,19.000000,25.000000,3.150000,3.110000,8.600000,4800.000000
50%,1.000000,97.000000,2414.000000,120.000000,95.000000,24.000000,30.000000,3.310000,3.290000,9.000000,5200.000000
75%,2.000000,102.400000,2935.000000,141.000000,116.000000,30.000000,34.000000,3.580000,3.410000,9.400000,5500.000000
max,3.000000,120.900000,4066.000000,326.000000,288.000000,49.000000,54.000000,3.940000,4.170000,23.000000,6600.000000


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train,y_train)

LinearRegression()

In [ ]:
print('Coefficients : ',model.coef_)
print('Intercept : ',model.intercept_)

Coefficients :  [ 2.30659098e+02 -3.29188420e+03  1.56253914e+02  2.06328463e+02
  3.23054402e-01  1.26800571e+02  2.13546610e+01 -4.46576839e+02
  2.46492635e+02 -5.42928318e+02 -3.56159775e+03  1.29833048e+02
  2.82950498e+00]
Intercept :  -22292.042841029157


In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('R-squared:', r2)

Mean Squared Error: 14306883.436636576
R-squared: 0.8187717919961792


In [ ]:
joblib.dump(model,"car_price_model.pkl")

['car_price_model.pkl']

In [ ]:
from google.colab import files

files.download('car_price_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>